In [6]:
import os
import graphviz
from collections import defaultdict, deque

# Function to load NFA from the input.txt file
def load_nfa(input_file):
    nfa = {
        'states': set(),
        'alphabet': set(),
        'transitions': defaultdict(dict),
        'start_state': None,
        'accept_states': set()
    }

    # Open the file and parse each line
    with open(input_file, 'r') as file:
        for line in file:
            line = line.strip()

            if line.startswith("States:"):
                # Extract states from the file (removing the braces and splitting by commas)
                nfa['states'] = set(line.split(":")[1].strip().strip('{}').split(", "))

            elif line.startswith("Alphabet:"):
                # Extract the alphabet (input symbols) from the file
                nfa['alphabet'] = set(line.split(":")[1].strip().strip('{}').split(", "))

            elif line.startswith("Start State:"):
                # Define the starting state
                nfa['start_state'] = line.split(":")[1].strip()

            elif line.startswith("Accept States:"):
                # Identify the accept states, removing any braces
                nfa['accept_states'] = set(line.split(":")[1].strip().strip('{}').split(", "))

            elif line.startswith("Transitions:"):
                # Parse the transitions, formatted as 'state -> symbol -> next_states'
                for transition in file:
                    transition = transition.strip()
                    if "->" in transition:
                        state, symbol, next_states = transition.split("->")
                        state = state.strip()
                        symbol = symbol.strip()
                        next_states = next_states.strip().split(", ")

                        # Store the transitions for the NFA
                        nfa['transitions'].setdefault(state, {}).setdefault(symbol, []).extend(next_states)
    
    return nfa

# Function to convert an NFA to a DFA using the Powerset Construction method
def nfa_to_dfa(nfa):
    dfa = {
        'states': set(),
        'alphabet': nfa['alphabet'],
        'transitions': defaultdict(dict),
        'start_state': None,
        'accept_states': set()
    }

    # The initial state of the DFA is the set containing the NFA's start state
    start_state = frozenset([nfa['start_state']])
    dfa['start_state'] = str(start_state)  # Convert set to string for naming consistency
    dfa['states'].add(str(start_state))

    # Initialize a queue to explore new DFA states
    queue = deque([start_state])
    visited = set([str(start_state)])

    while queue:
        current_dfa_state = queue.popleft()

        for symbol in nfa['alphabet']:
            next_nfa_states = set()
            # Determine which NFA states can be reached from current DFA state on each symbol
            for nfa_state in current_dfa_state:
                if symbol in nfa['transitions'].get(nfa_state, {}):
                    next_nfa_states.update(nfa['transitions'][nfa_state][symbol])

            # Create a new DFA state corresponding to the set of reachable NFA states
            next_dfa_state = frozenset(next_nfa_states)
            next_dfa_state_str = str(next_dfa_state)  # Use string representation for consistency

            if next_dfa_state:
                dfa['transitions'][str(current_dfa_state)][symbol] = next_dfa_state_str
                if next_dfa_state_str not in visited:
                    visited.add(next_dfa_state_str)
                    queue.append(next_dfa_state)
                    dfa['states'].add(next_dfa_state_str)

            # Mark a DFA state as an accept state if any corresponding NFA states are accept states
            if any(state in nfa['accept_states'] for state in next_nfa_states):
                dfa['accept_states'].add(next_dfa_state_str)

    return dfa

# Function to minimize a DFA
def minimize_dfa(dfa):
    # Initial partition: separate accepting and non-accepting states
    P = [dfa['accept_states'], dfa['states'] - dfa['accept_states']]
    W = [dfa['accept_states'].copy()]  # Subset of states that need further examination

    # Refine partitions based on transitions
    while W:
        A = W.pop()

        for symbol in dfa['alphabet']:
            # Identify states that can transition on 'symbol' to a state in 'A'
            X = {state for state in dfa['states'] if symbol in dfa['transitions'].get(state, {}) and dfa['transitions'][state][symbol] in A}

            for Y in P[:]:
                intersection = Y & X
                difference = Y - X

                if intersection and difference:
                    # Split partition Y into two: intersection and difference
                    P.remove(Y)
                    P.append(intersection)
                    P.append(difference)

                    # Add the smaller subset to W
                    if intersection in W:
                        W.append(difference)
                    else:
                        W.append(intersection)

    # Create minimized DFA
    minimized_dfa = {
        'states': set(),
        'alphabet': dfa['alphabet'],
        'transitions': defaultdict(dict),
        'start_state': None,
        'accept_states': set()
    }

    # Map old DFA states to new DFA states based on partitions
    state_map = {}
    for partition in P:
        new_state = str(frozenset(partition))  # Use string representation for consistency
        minimized_dfa['states'].add(new_state)

        for state in partition:
            state_map[state] = new_state
            if state == dfa['start_state']:
                minimized_dfa['start_state'] = new_state
            if state in dfa['accept_states']:
                minimized_dfa['accept_states'].add(new_state)

    # Define transitions for the minimized DFA
    for old_state, transitions in dfa['transitions'].items():
        new_state = state_map[old_state]
        for symbol, next_old_state in transitions.items():
            minimized_dfa['transitions'][new_state][symbol] = state_map[next_old_state]

    return minimized_dfa

# Function to create and render a graph representation of an automaton
def draw_graph(automaton, output_file):
    dot = graphviz.Digraph()
    
    # Set graph size and resolution
    dot.attr(size='100,200', dpi='800')
    
    # Add nodes with double circles for accept states
    for state in automaton.get('states', []):
        if state in automaton.get('accept_states', []):
            dot.node(state, shape='doublecircle')
        else:
            dot.node(state, shape='circle')
    
    # Add edges based on transitions
    for state, transitions in automaton.get('transitions', {}).items():
        for symbol, next_states in transitions.items():
            for next_state in next_states:
                dot.edge(state, next_state, label=symbol)
    
    # Save the graph as a PNG image
    dot.render(output_file, format='png', cleanup=True)

# Function to handle a single test case
def handle_testcase(testcase_dir):
    input_file = os.path.join(testcase_dir, 'input.txt')
    dfa_output_file = os.path.join(testcase_dir, 'dfa_image')
    minimized_dfa_output_file = os.path.join(testcase_dir, 'minimized_dfa_image')

    # Load NFA from the file
    nfa = load_nfa(input_file)
    
    # Convert the NFA to a DFA
    dfa = nfa_to_dfa(nfa)
    
    # Create the DFA graph image
    draw_graph(dfa, dfa_output_file)

    # Minimize the DFA
    minimized_dfa = minimize_dfa(dfa)
    
    # Create the minimized DFA graph image
    draw_graph(minimized_dfa, minimized_dfa_output_file)

# Process all test cases in the given directory
def handle_all_testcases(testcases_root):
    for testcase in os.listdir(testcases_root):
        testcase_dir = os.path.join(testcases_root, testcase)
        if os.path.isdir(testcase_dir):
            print(f"Processing {testcase_dir}...")
            handle_testcase(testcase_dir)

# Entry point for running the script
if __name__ == "__main__":
    handle_all_testcases('testcases')

Processing testcases\testcase-1...


dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.409587 to fit
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.409587 to fit


Processing testcases\testcase-2...


dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.814877 to fit


Processing testcases\testcase-3...


dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.714392 to fit
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.409587 to fit
